# Module 4.1: Linear Regression - 線性回歸

## 學習目標

完成這個 notebook 後，你將能夠：

1. 理解線性回歸的數學原理
2. 使用閉式解（Normal Equation）求解線性回歸
3. 使用梯度下降法求解線性回歸
4. 實作 L2 正則化（Ridge Regression）
5. 使用數值梯度檢查驗證你的梯度推導

## 背景知識

線性回歸是最基礎的監督式學習模型，用於預測連續值。雖然簡單，但理解它的數學推導對後續學習非常重要。

### 問題設定

給定訓練資料 $\{(x_i, y_i)\}_{i=1}^N$，其中：
- $x_i \in \mathbb{R}^D$ 是特徵向量
- $y_i \in \mathbb{R}$ 是目標值

我們假設 $y$ 與 $x$ 有線性關係：

$$\hat{y} = w^T x + b = \sum_{j=1}^D w_j x_j + b$$

目標是找到最佳的 $w$ 和 $b$ 使預測誤差最小。

---

## 參考資源

- Bishop PRML Ch. 3: Linear Models for Regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 設定 matplotlib
plt.rcParams['figure.figsize'] = [10, 6]

# 設定隨機種子
np.random.seed(42)

print("環境設定完成！")
print(f"NumPy 版本: {np.__version__}")

---

## Part 1: 損失函數 - Mean Squared Error (MSE)

### MSE 定義

均方誤差（Mean Squared Error）是最常用的回歸損失函數：

$$L(w, b) = \frac{1}{N} \sum_{i=1}^N (\hat{y}_i - y_i)^2 = \frac{1}{N} \sum_{i=1}^N (w^T x_i + b - y_i)^2$$

### 為什麼用 MSE？

1. **統計意義**：假設雜訊服從高斯分布時，最大化似然等價於最小化 MSE
2. **凸函數**：MSE 是凸函數，有唯一全域最小值
3. **可微分**：處處可微，方便優化

In [ ]:
def mse_loss(y_pred, y_true):
    """
    計算 Mean Squared Error
    
    Parameters
    ----------
    y_pred : np.ndarray, shape (N,)
        預測值
    y_true : np.ndarray, shape (N,)
        真實值
    
    Returns
    -------
    float
        MSE 損失值
    """
    N = len(y_true)
    return np.sum((y_pred - y_true) ** 2) / N


# 建立簡單測試資料
def generate_linear_data(n_samples=100, n_features=1, noise_std=0.5, seed=42):
    """
    生成線性回歸測試資料
    
    y = w^T x + b + noise
    """
    np.random.seed(seed)
    
    # 真實參數
    w_true = np.random.randn(n_features) * 2
    b_true = np.random.randn() * 2
    
    # 生成資料
    X = np.random.randn(n_samples, n_features) * 3
    y = X @ w_true + b_true + np.random.randn(n_samples) * noise_std
    
    return X, y, w_true, b_true


# 測試
print("=== 測試 MSE Loss ===")

X, y, w_true, b_true = generate_linear_data(n_samples=100, n_features=1)

print(f"真實參數: w = {w_true}, b = {b_true:.4f}")

# 用真實參數預測
y_pred_true = X @ w_true + b_true
print(f"用真實參數的 MSE: {mse_loss(y_pred_true, y):.4f}")

# 用隨機參數預測
y_pred_random = X @ np.random.randn(1) + np.random.randn()
print(f"用隨機參數的 MSE: {mse_loss(y_pred_random, y):.4f}")

# 視覺化（1D 情況）
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6, label='Data')
plt.plot(X, y_pred_true, 'r-', linewidth=2, label='True model')
plt.xlabel('x')
plt.ylabel('y')
plt.title('線性回歸測試資料')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## Part 2: 閉式解（Normal Equation）

### 矩陣形式

為了方便推導，我們把 bias 併入 weight 中。令 $\tilde{X}$ 是在 $X$ 前面加一行 1：

$$\tilde{X} = \begin{bmatrix} 1 & x_1^T \\ 1 & x_2^T \\ \vdots \\ 1 & x_N^T \end{bmatrix} \in \mathbb{R}^{N \times (D+1)}, \quad \tilde{w} = \begin{bmatrix} b \\ w \end{bmatrix} \in \mathbb{R}^{D+1}$$

則：
$$\hat{y} = \tilde{X} \tilde{w}$$

$$L(\tilde{w}) = \frac{1}{N} \|\tilde{X}\tilde{w} - y\|^2$$

### 求解

對 $\tilde{w}$ 微分並設為 0：

$$\frac{\partial L}{\partial \tilde{w}} = \frac{2}{N} \tilde{X}^T (\tilde{X}\tilde{w} - y) = 0$$

$$\tilde{X}^T \tilde{X} \tilde{w} = \tilde{X}^T y$$

$$\tilde{w}^* = (\tilde{X}^T \tilde{X})^{-1} \tilde{X}^T y$$

這就是 **Normal Equation**（正規方程）。

In [ ]:
def linear_regression_closed_form(X, y):
    """
    使用閉式解（Normal Equation）求解線性回歸
    
    Parameters
    ----------
    X : np.ndarray, shape (N, D)
        特徵矩陣
    y : np.ndarray, shape (N,)
        目標值
    
    Returns
    -------
    w : np.ndarray, shape (D,)
        權重
    b : float
        偏置
    """
    N, D = X.shape
    
    # Step 1: 在 X 前面加一行 1（用於 bias）
    # X_tilde shape: (N, D+1)
    X_tilde = np.hstack([np.ones((N, 1)), X])
    
    # Step 2: 計算 (X^T X)^{-1} X^T y
    # 使用 np.linalg.solve 比直接求逆更數值穩定
    # X^T X * w = X^T y
    XTX = X_tilde.T @ X_tilde
    XTy = X_tilde.T @ y
    
    # 解線性系統（比 np.linalg.inv 更穩定）
    w_tilde = np.linalg.solve(XTX, XTy)
    
    # Step 3: 分離 b 和 w
    b = w_tilde[0]
    w = w_tilde[1:]
    
    return w, b


# 測試閉式解
print("=== 測試閉式解 ===")

X, y, w_true, b_true = generate_linear_data(n_samples=100, n_features=3, noise_std=0.5)

print(f"真實參數:")
print(f"  w = {w_true}")
print(f"  b = {b_true:.4f}")

w_pred, b_pred = linear_regression_closed_form(X, y)

print(f"\n閉式解:")
print(f"  w = {w_pred}")
print(f"  b = {b_pred:.4f}")

# 計算預測誤差
y_pred = X @ w_pred + b_pred
print(f"\n訓練 MSE: {mse_loss(y_pred, y):.6f}")

# 參數誤差
print(f"\n參數誤差:")
print(f"  |w - w_true| = {np.linalg.norm(w_pred - w_true):.6f}")
print(f"  |b - b_true| = {abs(b_pred - b_true):.6f}")

### 閉式解什麼時候會失敗？

$X^T X$ 不可逆的情況：

1. **N < D**：樣本數小於特徵數（欠定系統）
2. **共線性**：特徵之間有線性相關（例如 $x_2 = 2x_1$）

解決方法：**正則化**

In [ ]:
# 示範 X^T X 不可逆的情況
print("=== 示範不可逆情況 ===")

# Case 1: N < D
print("\nCase 1: 樣本數 < 特徵數")
X_bad1 = np.random.randn(5, 10)  # 5 samples, 10 features
y_bad1 = np.random.randn(5)

try:
    w, b = linear_regression_closed_form(X_bad1, y_bad1)
    print(f"  成功，但結果可能不穩定")
except np.linalg.LinAlgError as e:
    print(f"  失敗: {e}")

# 檢查條件數
X_tilde = np.hstack([np.ones((5, 1)), X_bad1])
XTX = X_tilde.T @ X_tilde
cond_num = np.linalg.cond(XTX)
print(f"  X^T X 的條件數: {cond_num:.2e} (太大表示接近奇異)")

# Case 2: 共線性
print("\nCase 2: 共線性特徵")
X_base = np.random.randn(100, 2)
X_bad2 = np.hstack([X_base, X_base[:, 0:1] * 2])  # 第三個特徵是第一個的 2 倍
y_bad2 = np.random.randn(100)

X_tilde2 = np.hstack([np.ones((100, 1)), X_bad2])
XTX2 = X_tilde2.T @ X_tilde2
print(f"  X^T X 的行列式: {np.linalg.det(XTX2):.2e}")
print(f"  X^T X 的條件數: {np.linalg.cond(XTX2):.2e}")

---

## Part 3: 梯度下降法

### MSE 的梯度推導

$$L(w, b) = \frac{1}{N} \sum_{i=1}^N (w^T x_i + b - y_i)^2$$

令 $e_i = w^T x_i + b - y_i$（殘差），則：

$$\frac{\partial L}{\partial w} = \frac{2}{N} \sum_{i=1}^N e_i \cdot x_i = \frac{2}{N} X^T (Xw + b\mathbf{1} - y)$$

$$\frac{\partial L}{\partial b} = \frac{2}{N} \sum_{i=1}^N e_i = \frac{2}{N} \mathbf{1}^T (Xw + b\mathbf{1} - y)$$

其中 $\mathbf{1}$ 是全 1 向量。

### 梯度下降更新規則

$$w \leftarrow w - \eta \frac{\partial L}{\partial w}$$
$$b \leftarrow b - \eta \frac{\partial L}{\partial b}$$

In [ ]:
def compute_gradients(X, y, w, b):
    """
    計算 MSE 對 w 和 b 的梯度
    
    Parameters
    ----------
    X : np.ndarray, shape (N, D)
    y : np.ndarray, shape (N,)
    w : np.ndarray, shape (D,)
    b : float
    
    Returns
    -------
    dw : np.ndarray, shape (D,)
        ∂L/∂w
    db : float
        ∂L/∂b
    """
    N = X.shape[0]
    
    # 計算殘差 e = y_pred - y
    y_pred = X @ w + b
    error = y_pred - y  # shape (N,)
    
    # 梯度
    dw = (2 / N) * (X.T @ error)
    db = (2 / N) * np.sum(error)
    
    return dw, db


def linear_regression_gradient_descent(X, y, lr=0.01, n_iter=1000, verbose=True):
    """
    使用梯度下降求解線性回歸
    
    Parameters
    ----------
    X : np.ndarray, shape (N, D)
    y : np.ndarray, shape (N,)
    lr : float
        學習率
    n_iter : int
        迭代次數
    verbose : bool
        是否輸出訓練過程
    
    Returns
    -------
    w : np.ndarray, shape (D,)
    b : float
    history : dict
        訓練歷史（loss, w, b）
    """
    N, D = X.shape
    
    # 初始化參數
    w = np.zeros(D)
    b = 0.0
    
    # 記錄歷史
    history = {'loss': [], 'w': [], 'b': []}
    
    for i in range(n_iter):
        # 計算梯度
        dw, db = compute_gradients(X, y, w, b)
        
        # 更新參數
        w = w - lr * dw
        b = b - lr * db
        
        # 計算 loss
        y_pred = X @ w + b
        loss = mse_loss(y_pred, y)
        
        # 記錄
        history['loss'].append(loss)
        history['w'].append(w.copy())
        history['b'].append(b)
        
        if verbose and (i + 1) % (n_iter // 10) == 0:
            print(f"Iter {i+1:4d}: Loss = {loss:.6f}")
    
    return w, b, history


# 測試梯度下降
print("=== 測試梯度下降 ===")

X, y, w_true, b_true = generate_linear_data(n_samples=100, n_features=3, noise_std=0.5)

print(f"真實參數: w = {w_true}, b = {b_true:.4f}\n")

w_gd, b_gd, history = linear_regression_gradient_descent(X, y, lr=0.01, n_iter=1000)

print(f"\n梯度下降解:")
print(f"  w = {w_gd}")
print(f"  b = {b_gd:.4f}")

# 與閉式解比較
w_closed, b_closed = linear_regression_closed_form(X, y)
print(f"\n閉式解:")
print(f"  w = {w_closed}")
print(f"  b = {b_closed:.4f}")

print(f"\n梯度下降與閉式解的差異:")
print(f"  |w_gd - w_closed| = {np.linalg.norm(w_gd - w_closed):.8f}")
print(f"  |b_gd - b_closed| = {abs(b_gd - b_closed):.8f}")

In [ ]:
# 視覺化訓練過程
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss 曲線
axes[0].plot(history['loss'])
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('訓練 Loss 曲線')
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

# 參數收斂
w_history = np.array(history['w'])
for j in range(w_history.shape[1]):
    axes[1].plot(w_history[:, j], label=f'w[{j}]')
axes[1].axhline(y=w_true[0], color='r', linestyle='--', alpha=0.5, label='True w[0]')
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Parameter Value')
axes[1].set_title('參數收斂過程')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part 4: 數值梯度檢查

確認我們推導的解析梯度是正確的。使用有限差分近似：

$$\frac{\partial L}{\partial \theta_j} \approx \frac{L(\theta + \epsilon e_j) - L(\theta - \epsilon e_j)}{2\epsilon}$$

其中 $e_j$ 是第 $j$ 個基向量，$\epsilon$ 是小量（如 $10^{-5}$）。

In [ ]:
def numerical_gradient(f, x, eps=1e-5):
    """
    計算數值梯度
    
    Parameters
    ----------
    f : callable
        損失函數 f(x) -> scalar
    x : np.ndarray
        參數
    eps : float
        有限差分的步長
    
    Returns
    -------
    grad : np.ndarray
        數值梯度，與 x 形狀相同
    """
    grad = np.zeros_like(x)
    
    # 對每個維度計算數值梯度
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        old_value = x[idx]
        
        # f(x + eps)
        x[idx] = old_value + eps
        fx_plus = f(x)
        
        # f(x - eps)
        x[idx] = old_value - eps
        fx_minus = f(x)
        
        # 還原
        x[idx] = old_value
        
        # 中心差分
        grad[idx] = (fx_plus - fx_minus) / (2 * eps)
        
        it.iternext()
    
    return grad


def gradient_check(X, y, w, b, eps=1e-5):
    """
    檢查解析梯度與數值梯度是否一致
    """
    # 解析梯度
    dw_analytic, db_analytic = compute_gradients(X, y, w, b)
    
    # 數值梯度 for w
    def loss_w(w_test):
        y_pred = X @ w_test + b
        return mse_loss(y_pred, y)
    
    dw_numeric = numerical_gradient(loss_w, w.copy(), eps)
    
    # 數值梯度 for b
    def loss_b(b_test):
        y_pred = X @ w + b_test[0]
        return mse_loss(y_pred, y)
    
    db_numeric = numerical_gradient(loss_b, np.array([b]), eps)[0]
    
    # 比較
    print("=== 梯度檢查 ===")
    print(f"dw 解析梯度: {dw_analytic}")
    print(f"dw 數值梯度: {dw_numeric}")
    print(f"dw 相對誤差: {np.linalg.norm(dw_analytic - dw_numeric) / (np.linalg.norm(dw_analytic) + np.linalg.norm(dw_numeric) + 1e-8):.2e}")
    print()
    print(f"db 解析梯度: {db_analytic:.6f}")
    print(f"db 數值梯度: {db_numeric:.6f}")
    print(f"db 相對誤差: {abs(db_analytic - db_numeric) / (abs(db_analytic) + abs(db_numeric) + 1e-8):.2e}")
    
    # 判定
    w_ok = np.allclose(dw_analytic, dw_numeric, rtol=1e-4, atol=1e-6)
    b_ok = np.allclose(db_analytic, db_numeric, rtol=1e-4, atol=1e-6)
    
    print(f"\n梯度檢查通過: {w_ok and b_ok}")
    return w_ok and b_ok


# 執行梯度檢查
X, y, _, _ = generate_linear_data(n_samples=50, n_features=3)
w_test = np.random.randn(3)
b_test = np.random.randn()

gradient_check(X, y, w_test, b_test)

---

## Part 5: 正則化（Ridge Regression）

### L2 正則化

在損失函數中加入權重的 L2 範數：

$$L_{\text{ridge}}(w, b) = \frac{1}{N} \sum_{i=1}^N (\hat{y}_i - y_i)^2 + \lambda \|w\|^2$$

其中 $\lambda > 0$ 是正則化強度。

### 為什麼需要正則化？

1. **防止過擬合**：限制權重大小，減少模型複雜度
2. **解決不可逆問題**：$X^T X + \lambda I$ 一定可逆（$\lambda > 0$ 時）

### 閉式解

$$\tilde{w}^* = (\tilde{X}^T \tilde{X} + \lambda I')^{-1} \tilde{X}^T y$$

注意：$I'$ 的第一個對角元素是 0（不對 bias 正則化）。

In [ ]:
def ridge_regression_closed_form(X, y, reg=0.0):
    """
    使用閉式解求解 Ridge 回歸
    
    Parameters
    ----------
    X : np.ndarray, shape (N, D)
    y : np.ndarray, shape (N,)
    reg : float
        正則化強度 λ
    
    Returns
    -------
    w : np.ndarray, shape (D,)
    b : float
    """
    N, D = X.shape
    
    # 加入 bias 項
    X_tilde = np.hstack([np.ones((N, 1)), X])
    
    # 建立正則化矩陣
    # 不對 bias 正則化，所以第一個對角元素是 0
    reg_matrix = reg * np.eye(D + 1)
    reg_matrix[0, 0] = 0  # 不正則化 bias
    
    # 閉式解
    XTX = X_tilde.T @ X_tilde + reg_matrix
    XTy = X_tilde.T @ y
    
    w_tilde = np.linalg.solve(XTX, XTy)
    
    b = w_tilde[0]
    w = w_tilde[1:]
    
    return w, b


def ridge_regression_gradient_descent(X, y, reg=0.0, lr=0.01, n_iter=1000):
    """
    使用梯度下降求解 Ridge 回歸
    
    梯度：
    ∂L/∂w = (2/N) X^T (Xw + b - y) + 2λw
    ∂L/∂b = (2/N) 1^T (Xw + b - y)  (bias 不正則化)
    """
    N, D = X.shape
    
    w = np.zeros(D)
    b = 0.0
    
    history = {'loss': []}
    
    for i in range(n_iter):
        # 預測
        y_pred = X @ w + b
        error = y_pred - y
        
        # 梯度
        dw = (2 / N) * (X.T @ error) + 2 * reg * w
        db = (2 / N) * np.sum(error)
        
        # 更新
        w = w - lr * dw
        b = b - lr * db
        
        # Loss（含正則化項）
        loss = mse_loss(y_pred, y) + reg * np.sum(w ** 2)
        history['loss'].append(loss)
    
    return w, b, history


# 測試正則化
print("=== 測試 Ridge Regression ===")

# 建立有共線性的資料
np.random.seed(42)
X_base = np.random.randn(100, 5)
X_collinear = np.hstack([X_base, X_base[:, 0:1] * 2 + 0.01 * np.random.randn(100, 1)])  # 近似共線
y_collinear = X_base @ np.array([1, 2, 3, 4, 5]) + np.random.randn(100) * 0.5

print("資料有近似共線性...")

# 不同正則化強度
regs = [0, 0.01, 0.1, 1.0, 10.0]

print(f"\n{'λ':>10} {'||w||':>12} {'Train MSE':>12}")
print("-" * 36)

for reg in regs:
    w, b = ridge_regression_closed_form(X_collinear, y_collinear, reg)
    y_pred = X_collinear @ w + b
    mse = mse_loss(y_pred, y_collinear)
    w_norm = np.linalg.norm(w)
    print(f"{reg:>10.2f} {w_norm:>12.4f} {mse:>12.6f}")

print("\n觀察：正則化越強，||w|| 越小，但 MSE 可能增加（bias-variance trade-off）")

In [ ]:
# 視覺化正則化效果
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 權重大小 vs 正則化強度
regs = np.logspace(-3, 2, 50)
w_norms = []
mses = []

for reg in regs:
    w, b = ridge_regression_closed_form(X_collinear, y_collinear, reg)
    w_norms.append(np.linalg.norm(w))
    y_pred = X_collinear @ w + b
    mses.append(mse_loss(y_pred, y_collinear))

axes[0].semilogx(regs, w_norms)
axes[0].set_xlabel('λ (正則化強度)')
axes[0].set_ylabel('||w||')
axes[0].set_title('正則化強度 vs 權重大小')
axes[0].grid(True, alpha=0.3)

axes[1].semilogx(regs, mses)
axes[1].set_xlabel('λ (正則化強度)')
axes[1].set_ylabel('Training MSE')
axes[1].set_title('正則化強度 vs 訓練誤差')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 練習題

### 練習 1：實作多項式回歸

**任務**：使用特徵轉換實作多項式回歸

$$\hat{y} = w_0 + w_1 x + w_2 x^2 + ... + w_d x^d$$

**提示**：
- 建立特徵矩陣 $[1, x, x^2, ..., x^d]$
- 然後套用一般的線性回歸

In [ ]:
# 練習 1 解答

def polynomial_features(x, degree):
    """
    建立多項式特徵
    
    Parameters
    ----------
    x : np.ndarray, shape (N,)
        原始特徵
    degree : int
        多項式次數
    
    Returns
    -------
    X_poly : np.ndarray, shape (N, degree+1)
        多項式特徵 [1, x, x^2, ..., x^degree]
    """
    N = len(x)
    X_poly = np.zeros((N, degree + 1))
    
    for d in range(degree + 1):
        X_poly[:, d] = x ** d
    
    return X_poly


def polynomial_regression(x, y, degree, reg=0.0):
    """
    多項式回歸
    
    Parameters
    ----------
    x : np.ndarray, shape (N,)
    y : np.ndarray, shape (N,)
    degree : int
    reg : float
        正則化強度
    
    Returns
    -------
    coefficients : np.ndarray, shape (degree+1,)
        多項式係數 [w_0, w_1, ..., w_d]
    """
    # 建立多項式特徵（不包含常數項，因為 ridge_regression 會加）
    X_poly = polynomial_features(x, degree)[:, 1:]  # 去掉常數項
    
    # 用 Ridge regression 擬合
    w, b = ridge_regression_closed_form(X_poly, y, reg)
    
    # 組合係數
    coefficients = np.concatenate([[b], w])
    
    return coefficients


# 測試多項式回歸
print("=== 測試多項式回歸 ===")

# 生成非線性資料
np.random.seed(42)
x_train = np.sort(np.random.uniform(-3, 3, 30))
y_train = np.sin(x_train) + np.random.randn(30) * 0.2  # 真實函數是 sin(x)

# 測試不同次數的多項式
x_test = np.linspace(-3.5, 3.5, 100)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

degrees = [1, 2, 3, 5, 9, 15]

for ax, degree in zip(axes, degrees):
    # 擬合
    coeffs = polynomial_regression(x_train, y_train, degree, reg=0.0)
    
    # 預測
    X_test_poly = polynomial_features(x_test, degree)
    y_pred = X_test_poly @ coeffs
    
    # 訓練 MSE
    X_train_poly = polynomial_features(x_train, degree)
    y_train_pred = X_train_poly @ coeffs
    mse = mse_loss(y_train_pred, y_train)
    
    # 繪圖
    ax.scatter(x_train, y_train, c='blue', alpha=0.6, label='Training data')
    ax.plot(x_test, np.sin(x_test), 'g--', linewidth=2, label='True function')
    ax.plot(x_test, y_pred, 'r-', linewidth=2, label=f'Degree {degree}')
    ax.set_title(f'Degree {degree}, MSE={mse:.4f}')
    ax.set_ylim(-2, 2)
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.suptitle('多項式回歸：不同次數的擬合效果', fontsize=14)
plt.tight_layout()
plt.show()

print("觀察：次數太高會過擬合（在訓練範圍外預測不穩定）")

### 練習 2：正則化對多項式回歸的影響

**任務**：觀察正則化如何幫助減少高次多項式的過擬合

In [ ]:
# 練習 2 解答

print("=== 正則化對高次多項式的影響 ===")

degree = 15  # 高次多項式容易過擬合
regs = [0, 0.0001, 0.001, 0.01, 0.1, 1.0]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, reg in zip(axes, regs):
    # 擬合
    coeffs = polynomial_regression(x_train, y_train, degree, reg=reg)
    
    # 預測
    X_test_poly = polynomial_features(x_test, degree)
    y_pred = X_test_poly @ coeffs
    
    # 訓練 MSE
    X_train_poly = polynomial_features(x_train, degree)
    y_train_pred = X_train_poly @ coeffs
    mse = mse_loss(y_train_pred, y_train)
    
    # 係數大小
    coeff_norm = np.linalg.norm(coeffs)
    
    # 繪圖
    ax.scatter(x_train, y_train, c='blue', alpha=0.6, label='Training data')
    ax.plot(x_test, np.sin(x_test), 'g--', linewidth=2, label='True function')
    ax.plot(x_test, y_pred, 'r-', linewidth=2, label=f'λ={reg}')
    ax.set_title(f'λ={reg}\nMSE={mse:.4f}, ||w||={coeff_norm:.2f}')
    ax.set_ylim(-2, 2)
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.suptitle(f'Degree {degree} 多項式回歸：正則化的影響', fontsize=14)
plt.tight_layout()
plt.show()

print("觀察：適當的正則化可以讓高次多項式也有好的泛化能力")

### 練習 3：Mini-batch 梯度下降

**任務**：實作 Mini-batch SGD 來加速大資料集的訓練

**提示**：
- 每次迭代只用一個 batch 的資料計算梯度
- Batch size 通常取 32, 64, 128 等

In [ ]:
# 練習 3 解答

def linear_regression_minibatch_sgd(X, y, lr=0.01, n_epochs=100, batch_size=32, verbose=True):
    """
    使用 Mini-batch SGD 求解線性回歸
    
    Parameters
    ----------
    X : np.ndarray, shape (N, D)
    y : np.ndarray, shape (N,)
    lr : float
        學習率
    n_epochs : int
        訓練 epochs 數
    batch_size : int
        Batch 大小
    verbose : bool
        是否輸出訓練過程
    
    Returns
    -------
    w : np.ndarray, shape (D,)
    b : float
    history : dict
    """
    N, D = X.shape
    
    # 初始化參數
    w = np.zeros(D)
    b = 0.0
    
    history = {'loss': []}
    
    # 每個 epoch 的 batch 數
    n_batches = (N + batch_size - 1) // batch_size
    
    for epoch in range(n_epochs):
        # 打亂資料順序
        indices = np.random.permutation(N)
        X_shuffled = X[indices]
        y_shuffled = y[indices]
        
        epoch_loss = 0.0
        
        for batch_idx in range(n_batches):
            # 取出 batch
            start = batch_idx * batch_size
            end = min(start + batch_size, N)
            X_batch = X_shuffled[start:end]
            y_batch = y_shuffled[start:end]
            
            # 計算梯度（在 batch 上）
            dw, db = compute_gradients(X_batch, y_batch, w, b)
            
            # 更新參數
            w = w - lr * dw
            b = b - lr * db
        
        # 計算整個訓練集的 loss（用於監控）
        y_pred = X @ w + b
        loss = mse_loss(y_pred, y)
        history['loss'].append(loss)
        
        if verbose and (epoch + 1) % (n_epochs // 10) == 0:
            print(f"Epoch {epoch+1:4d}: Loss = {loss:.6f}")
    
    return w, b, history


# 測試 Mini-batch SGD
print("=== 測試 Mini-batch SGD ===")

# 生成較大的資料集
X_large, y_large, w_true, b_true = generate_linear_data(n_samples=1000, n_features=10, noise_std=0.5)

print(f"資料大小: {X_large.shape}")
print(f"真實參數: w = {w_true[:3]}..., b = {b_true:.4f}\n")

# Mini-batch SGD
import time

start = time.time()
w_sgd, b_sgd, history_sgd = linear_regression_minibatch_sgd(
    X_large, y_large, lr=0.01, n_epochs=100, batch_size=32
)
time_sgd = time.time() - start

print(f"\nMini-batch SGD 完成 (耗時: {time_sgd:.3f}s)")

# 與閉式解比較
start = time.time()
w_closed, b_closed = linear_regression_closed_form(X_large, y_large)
time_closed = time.time() - start

print(f"閉式解完成 (耗時: {time_closed:.3f}s)")

# 比較結果
y_pred_sgd = X_large @ w_sgd + b_sgd
y_pred_closed = X_large @ w_closed + b_closed

print(f"\nMini-batch SGD MSE: {mse_loss(y_pred_sgd, y_large):.6f}")
print(f"閉式解 MSE: {mse_loss(y_pred_closed, y_large):.6f}")

# 視覺化訓練過程
plt.figure(figsize=(10, 5))
plt.plot(history_sgd['loss'])
plt.axhline(y=mse_loss(y_pred_closed, y_large), color='r', linestyle='--', label='Closed-form solution')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Mini-batch SGD 訓練曲線')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## 整合練習：完整的線性回歸類別

In [ ]:
class LinearRegression:
    """
    線性回歸模型
    
    支援閉式解和梯度下降兩種方法，以及 L2 正則化
    
    Parameters
    ----------
    regularization : float
        L2 正則化強度（預設 0，即不正則化）
    
    Attributes
    ----------
    w : np.ndarray
        權重向量
    b : float
        偏置
    history : dict
        訓練歷史（僅梯度下降法）
    """
    
    def __init__(self, regularization=0.0):
        self.reg = regularization
        self.w = None
        self.b = None
        self.history = None
    
    def fit_closed_form(self, X, y):
        """
        使用閉式解（Normal Equation）擬合
        
        w* = (X^T X + λI)^{-1} X^T y
        """
        N, D = X.shape
        
        # 加入 bias 項
        X_tilde = np.hstack([np.ones((N, 1)), X])
        
        # 正則化矩陣（不對 bias 正則化）
        reg_matrix = self.reg * np.eye(D + 1)
        reg_matrix[0, 0] = 0
        
        # 閉式解
        XTX = X_tilde.T @ X_tilde + reg_matrix
        XTy = X_tilde.T @ y
        
        w_tilde = np.linalg.solve(XTX, XTy)
        
        self.b = w_tilde[0]
        self.w = w_tilde[1:]
        
        return self
    
    def fit_gradient_descent(self, X, y, lr=0.01, n_iter=1000, verbose=False):
        """
        使用梯度下降擬合
        """
        N, D = X.shape
        
        # 初始化
        self.w = np.zeros(D)
        self.b = 0.0
        self.history = {'loss': []}
        
        for i in range(n_iter):
            # 預測
            y_pred = X @ self.w + self.b
            error = y_pred - y
            
            # 梯度（含正則化）
            dw = (2 / N) * (X.T @ error) + 2 * self.reg * self.w
            db = (2 / N) * np.sum(error)
            
            # 更新
            self.w = self.w - lr * dw
            self.b = self.b - lr * db
            
            # 記錄 loss
            loss = np.mean(error ** 2) + self.reg * np.sum(self.w ** 2)
            self.history['loss'].append(loss)
            
            if verbose and (i + 1) % (n_iter // 10) == 0:
                print(f"Iter {i+1}: Loss = {loss:.6f}")
        
        return self
    
    def predict(self, X):
        """
        預測
        """
        if self.w is None:
            raise ValueError("Model not fitted. Call fit_* first.")
        return X @ self.w + self.b
    
    def score(self, X, y):
        """
        計算 R² 分數
        
        R² = 1 - SS_res / SS_tot
        """
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        return 1 - ss_res / ss_tot


# 測試完整類別
print("=== 測試 LinearRegression 類別 ===")

X, y, w_true, b_true = generate_linear_data(n_samples=100, n_features=5)

# 閉式解
model_closed = LinearRegression(regularization=0.01)
model_closed.fit_closed_form(X, y)

print(f"閉式解 R²: {model_closed.score(X, y):.6f}")

# 梯度下降
model_gd = LinearRegression(regularization=0.01)
model_gd.fit_gradient_descent(X, y, lr=0.01, n_iter=2000, verbose=False)

print(f"梯度下降 R²: {model_gd.score(X, y):.6f}")

# 參數比較
print(f"\n參數差異: |w_closed - w_gd| = {np.linalg.norm(model_closed.w - model_gd.w):.8f}")

print("\nLinearRegression 類別測試完成！")

---

## 總結

### 本 Notebook 涵蓋的內容

1. **MSE 損失函數**：$L = \frac{1}{N} \sum (\hat{y} - y)^2$

2. **閉式解（Normal Equation）**：
   - $w^* = (X^T X)^{-1} X^T y$
   - 一次計算得到最優解
   - 但 $X^T X$ 可能不可逆

3. **梯度下降**：
   - 梯度：$\frac{\partial L}{\partial w} = \frac{2}{N} X^T (Xw + b - y)$
   - 迭代更新：$w \leftarrow w - \eta \nabla_w L$
   - Mini-batch SGD 加速大資料訓練

4. **正則化（Ridge）**：
   - $L_{ridge} = L_{MSE} + \lambda \|w\|^2$
   - 閉式解：$(X^T X + \lambda I)^{-1} X^T y$
   - 解決不可逆和過擬合問題

5. **數值梯度檢查**：
   - 用有限差分驗證解析梯度
   - 確保實作正確

### 關鍵要點

1. **公式 → 程式碼**：理解數學推導，然後用向量化實作
2. **梯度檢查**：永遠驗證你的梯度實作
3. **正則化**：在實際應用中幾乎總是需要的

### 下一步

在下一個 notebook 中，我們將學習 **Logistic Regression**（邏輯回歸），用於分類問題。